In [ ]:
!rm -r sample_data

rm: cannot remove 'sample_data': No such file or directory


In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-hxrvhf1_
  Running command git clone -q https://github.com/qubvel/segmentation_models /tmp/pip-req-build-hxrvhf1_
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 3.2 MB/s 
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-py3-none-any.whl size=33810 sha256=ce3f43261e580515086432d91f5c4f7906e6ebb22f2d19ecdedea85d0ca49b1a
  Stored in directory: /tmp/pip-ephem-wheel-cache-10ubpvs8/wheels/02/cd/18/61c0bbb8766acfec68f9d20618886b7b38dfeeb95865b6ba00
Successfully built segmentation-models


In [ ]:
!unzip /content/drive/MyDrive/Segmentation/dataSet/ph2dataset.zip

Archive:  /content/drive/MyDrive/Segmentation/dataSet/ph2dataset.zip
  inflating: ph2_dataset/trainx/IMD002.bmp  
  inflating: ph2_dataset/trainx/IMD003.bmp  
  inflating: ph2_dataset/trainx/IMD004.bmp  
  inflating: ph2_dataset/trainx/IMD006.bmp  
  inflating: ph2_dataset/trainx/IMD008.bmp  
  inflating: ph2_dataset/trainx/IMD009.bmp  
  inflating: ph2_dataset/trainx/IMD010.bmp  
  inflating: ph2_dataset/trainx/IMD013.bmp  
  inflating: ph2_dataset/trainx/IMD014.bmp  
  inflating: ph2_dataset/trainx/IMD015.bmp  
  inflating: ph2_dataset/trainx/IMD016.bmp  
  inflating: ph2_dataset/trainx/IMD017.bmp  
  inflating: ph2_dataset/trainx/IMD018.bmp  
  inflating: ph2_dataset/trainx/IMD019.bmp  
  inflating: ph2_dataset/trainx/IMD020.bmp  
  inflating: ph2_dataset/trainx/IMD021.bmp  
  inflating: ph2_dataset/trainx/IMD022.bmp  
  inflating: ph2_dataset/trainx/IMD023.bmp  
  inflating: ph2_dataset/trainx/IMD024.bmp  
  inflating: ph2_dataset/trainx/IMD025.bmp  
  inflating: ph2_dataset/trainx

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam

In [ ]:
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y


In [ ]:

# augmenting data
from albumentations import CenterCrop, RandomRotate90, HorizontalFlip, VerticalFlip
from tqdm import tqdm

def augment(images,masks,save_path,augment_data=True):
  H=256
  W=256
  for x,y in tqdm(zip(images,masks),total=len(images)):
    name=x.split('/')[-1].split(".")
    img_name=name[0]
    img_ext=name[1]

    name=x.split('/')[-1].split(".")
    mask_name=name[0]
    mask_ext=name[1]

    x=cv2.imread(x,cv2.IMREAD_COLOR)
    y=cv2.imread(y,cv2.IMREAD_COLOR)
    #print(x.shape)
    #print(y.shape)

    if augment_data==True:
      # Randome rotate
      aug=RandomRotate90(p=0.8)
      augmented=aug(image=x,mask=y)
      x2=augmented['image']
      y2=augmented['mask']
      # vertical Flip
      aug=VerticalFlip(p=0.6)
      augmented=aug(image=x,mask=y)
      x3=augmented['image']
      y3=augmented['mask']
      # Horizontal Flip Flip
      aug=HorizontalFlip(p=0.7)
      augmented=aug(image=x,mask=y)
      x4=augmented['image']
      y4=augmented['mask']
      #Save file
      save_images=[x,x2,x3,x4]
      save_masks=[y,y2,y3,y4]
    else:
      save_images=[x]
      save_masks=[y]
    idx=0
    for i,m in zip(save_images,save_masks):
      i=cv2.resize(i,(W,H))
      m=cv2.resize(m,(W,H))
      tmp_img_name=f"{img_name}_{idx}.{img_ext}"
      tmp_mask_name=f"{mask_name}_{idx}.{mask_ext}"
      image_path=os.path.join(save_path,"image",tmp_img_name)
      mask_path=os.path.join(save_path,"mask",tmp_mask_name)
      cv2.imwrite(image_path,i)
      cv2.imwrite(mask_path,m)
      idx+=1

In [ ]:
create_dir("augDataset")
create_dir("augDataset/image")
create_dir("augDataset/mask")

In [ ]:
path='/content/ph2_dataset'
images=sorted(glob(os.path.join(path,"trainx/*")))
masks=sorted(glob(os.path.join(path,"trainy/*")))
print(f"Orignal Image: {len(images)}")
print(f"Orignal Masks: {len(masks)}")

Orignal Image: 200
Orignal Masks: 200


In [ ]:
# augmenting PH2 DataSet
augment(images,masks,'/content/augDataset',True)
path='/content/augDataset'
images=sorted(glob(os.path.join(path,"image/*")))
masks=sorted(glob(os.path.join(path,"mask/*")))
print(f"Augmented Image: {len(images)}")
print(f"Augmented Masks: {len(masks)}")

100%|██████████| 200/200 [00:04<00:00, 40.42it/s]

Augmented Image: 800
Augmented Masks: 800


In [ ]:
# remove unwanted folders
!rm -r /content/ph2_dataset
!rm -r /content/trainx
!rm -r /content/trainy

In [ ]:
def load_spilt_data(dataset_path, split=0.2):
    images = sorted(glob(os.path.join(dataset_path, "image", "*.bmp")))
    masks = sorted(glob(os.path.join(dataset_path, "mask", "*.bmp")))
    #print(type(images))
    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [ ]:
""" Folder For saving result"""
create_dir("Files")

In [ ]:
# seeding 

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
""" Hyperparameters """
batch_size = 16
lr = 0.0001 
num_epoch = 50

In [ ]:
""" Dataset : 60/20/20 """
dataset_path = "/content/augDataset"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_spilt_data(dataset_path)

In [ ]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 480 - 480
Valid: 160 - 160
Test: 160 - 160


In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

In [ ]:
train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

In [ ]:
if len(train_x) % batch_size != 0:
  train_steps += 1

if len(valid_x) % batch_size != 0:
  valid_steps += 1
print(f"Training Steps: {train_steps}")
print(f"Validation Steps: {valid_steps}")

Training Steps: 30
Validation Steps: 10


In [ ]:
import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
# metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
metrics = [dice_coef, iou, Recall(), Precision()]

In [ ]:
from segmentation_models import Unet
model1 =Unet(backbone_name='vgg16',input_shape=[256,256,3],encoder_weights='imagenet',activation='sigmoid')
model2 =Unet(backbone_name='resnet34',input_shape=[256,256,3],encoder_weights='imagenet',activation='sigmoid')
model3 =Unet(backbone_name='inceptionv3',input_shape=[256,256,3],encoder_weights='imagenet',activation='sigmoid')

87924736/87910968 [==============================] - 1s 0us/step


VGG19 Model 

In [ ]:
model1.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 256, 256, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [ ]:
model_path="/content/Files/VGG_model1.h5"
csv_path="/content/Files/model_1.csv"
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

In [ ]:

model1.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.5580 - dice_coef: 0.5659 - iou: 0.3989 - recall: 0.8946 - precision: 0.5590
Epoch 1: val_loss improved from inf to 0.57090, saving model to /content/Files/VGG_model1.h5
30/30 [==============================] - 57s 926ms/step - loss: 0.5580 - dice_coef: 0.5659 - iou: 0.3989 - recall: 0.8946 - precision: 0.5590 - val_loss: 0.5709 - val_dice_coef: 0.5868 - val_iou: 0.4164 - val_recall: 0.9418 - val_precision: 0.4816 - lr: 1.0000e-04
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.3354 - dice_coef: 0.6782 - iou: 0.5152 - recall: 0.8980 - precision: 0.8634
Epoch 2: val_loss improved from 0.57090 to 0.37478, saving model to /content/Files/VGG_model1.h5
30/30 [==============================] - 27s 896ms/step - loss: 0.3354 - dice_coef: 0.6782 - iou: 0.5152 - recall: 0.8980 - precision: 0.8634 - val_loss: 0.3748 - val_dice_coef: 0.6519 - val_iou: 0.4844 - val_recall: 0.8706 - val_precision: 0.7617 - lr:

In [ ]:
model1.save('VGG_model1.hdf5')

In [ ]:
model2.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 256, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 128, 128, 64  9408        ['zero_padding2d[0][0]']   

In [ ]:
model_path="/content/Files/Resnet_model2.h5"
csv_path="/content/Files/model_2.csv"
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [ ]:
model2.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.4721 - dice_coef: 0.4938 - iou: 0.3330 - recall: 0.6181 - precision: 0.8651
Epoch 1: val_loss improved from inf to 0.64700, saving model to /content/Files/Resnet_model2.h5
30/30 [==============================] - 29s 643ms/step - loss: 0.4721 - dice_coef: 0.4938 - iou: 0.3330 - recall: 0.6181 - precision: 0.8651 - val_loss: 0.6470 - val_dice_coef: 0.3780 - val_iou: 0.2334 - val_recall: 0.1229 - val_precision: 0.5731 - lr: 1.0000e-04
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.2500 - dice_coef: 0.7050 - iou: 0.5467 - recall: 0.8726 - precision: 0.9206
Epoch 2: val_loss improved from 0.64700 to 0.55393, saving model to /content/Files/Resnet_model2.h5
30/30 [==============================] - 19s 631ms/step - loss: 0.2500 - dice_coef: 0.7050 - iou: 0.5467 - recall: 0.8726 - precision: 0.9206 - val_loss: 0.5539 - val_dice_coef: 0.4063 - val_iou: 0.2550 - val_recall: 0.3115 - val_precision: 0.8433

In [ ]:
model2.save('Resnet_model2.hdf5')

In [ ]:
model3.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
model3.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [ ]:
model_path="/content/Files/inception_model3.h5"
csv_path="/content/Files/model_3.csv"
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

In [ ]:
model3.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.4586 - dice_coef: 0.5283 - iou: 0.3635 - recall: 0.5547 - precision: 0.6952
Epoch 1: val_loss improved from inf to 0.45081, saving model to /content/Files/inception_model3.h5
30/30 [==============================] - 44s 950ms/step - loss: 0.4586 - dice_coef: 0.5283 - iou: 0.3635 - recall: 0.5547 - precision: 0.6952 - val_loss: 0.4508 - val_dice_coef: 0.5388 - val_iou: 0.3690 - val_recall: 0.8126 - val_precision: 0.6773 - lr: 1.0000e-04
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.2590 - dice_coef: 0.6939 - iou: 0.5334 - recall: 0.9321 - precision: 0.9029
Epoch 2: val_loss improved from 0.45081 to 0.29631, saving model to /content/Files/inception_model3.h5
30/30 [==============================] - 27s 891ms/step - loss: 0.2590 - dice_coef: 0.6939 - iou: 0.5334 - recall: 0.9321 - precision: 0.9029 - val_loss: 0.2963 - val_dice_coef: 0.7162 - val_iou: 0.5581 - val_recall: 0.9019 - val_precision: 

In [ ]:
model3.save('inception_model3.hdf5')

In [ ]:
from keras.models import load_model

#Set compile=False as we are not loading it for training, only for prediction.
model1 = load_model('/content/VGG_model1.hdf5', compile=False)
model2 = load_model('/content/drive/MyDrive/Segmentation/Ensemble_segmentation_Model/Ph2_model/Resnet_model2.hdf5', compile=False)
model3 = load_model('/content/inception_model3.hdf5', compile=False)

In [ ]:
def read_test_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                                ## (1, 256, 256, 3)


def read_test_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)                    ## (256, 256)
    return ori_x, x

In [ ]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
SCORE = []

In [ ]:

weights = [0.6, 0.6, 0.6]

for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
  name = x.split("/")[-1]
  """ Exctracting the image name """
  """ Read the image and mask """
  ori_x, x = read_test_image(x)
  ori_y, y = read_test_mask(y)
  """ Predicting the mask with Model-1 """
  y_pred_1_en = model1.predict(x)[0] > 0.5
  y_pred_1 = np.squeeze(y_pred_1_en, axis=-1)
  y_pred_1 = y_pred_1.astype(np.int32)

  """ Predicting the mask with Model-2 """
  y_pred_2_en = model2.predict(x)[0] > 0.5
  y_pred_2 = np.squeeze(y_pred_2_en, axis=-1)
  y_pred_2 = y_pred_2.astype(np.int32)

  """ Predicting the mask with Model-3 """
  y_pred_3_en = model3.predict(x)[0] > 0.5
  y_pred_3 = np.squeeze(y_pred_3_en, axis=-1)
  y_pred_3 = y_pred_3.astype(np.int32)
  
  """ Predicting the mask with ensemble-Model """
  y_pred_ensem=np.array([y_pred_1_en,y_pred_2_en,y_pred_3_en])
  y_pred_ensem=np.tensordot(y_pred_ensem,weights,axes=((0),(0)))>0.5
  y_pred_ensem = np.squeeze(y_pred_ensem, axis=-1)
  y_pred_ensem = y_pred_ensem.astype(np.int32)

  """ Flatten the array """
  # for model 1
  y = y.flatten()
  y_pred_1 = y_pred_1.flatten()

  """ Calculating metrics values """
  acc_value_1 = dice(y_pred_1, y)
  f1_value_1 = f1_score(y, y_pred_1, labels=[0, 1], average="binary")
  jac_value_1 = jaccard_score(y, y_pred_1, labels=[0, 1], average="binary")
  recall_value_1 = recall_score(y, y_pred_1, labels=[0, 1], average="binary")
  precision_value_1 = precision_score(y, y_pred_1, labels=[0, 1], average="binary")
  
  # for model 2 
  y_pred_2 = y_pred_2.flatten()

  """ Calculating metrics values """
  acc_value_2 = dice(y_pred_2, y)
  f1_value_2 = f1_score(y, y_pred_2, labels=[0, 1], average="binary")
  jac_value_2 = jaccard_score(y, y_pred_2, labels=[0, 1], average="binary")
  recall_value_2 = recall_score(y, y_pred_2, labels=[0, 1], average="binary")
  precision_value_2 = precision_score(y, y_pred_2, labels=[0, 1], average="binary")

  # for model 3
  
  y_pred_3 = y_pred_3.flatten()

  """ Calculating metrics values """
  acc_value_3 = dice(y_pred_3, y)
  f1_value_3 = f1_score(y, y_pred_3, labels=[0, 1], average="binary")
  jac_value_3 = jaccard_score(y, y_pred_3, labels=[0, 1], average="binary")
  recall_value_3 = recall_score(y, y_pred_3, labels=[0, 1], average="binary")
  precision_value_3 = precision_score(y, y_pred_3, labels=[0, 1], average="binary")

  # model 4
  y_pred_ensem = y_pred_ensem.flatten()

  """ Calculating metrics values """
  acc_value_4 = dice(y_pred_ensem, y)
  f1_value_4 = f1_score(y, y_pred_ensem, labels=[0, 1], average="binary")
  jac_value_4 = jaccard_score(y, y_pred_ensem, labels=[0, 1], average="binary")
  recall_value_4 = recall_score(y, y_pred_ensem, labels=[0, 1], average="binary")
  precision_value_4 = precision_score(y, y_pred_ensem, labels=[0, 1], average="binary")

  SCORE.append([name, acc_value_1, f1_value_1, jac_value_1, recall_value_1, precision_value_1,
                acc_value_2, f1_value_2, jac_value_2, recall_value_2, precision_value_2
               , acc_value_3, f1_value_3, jac_value_3, recall_value_3, precision_value_3
               , acc_value_4, f1_value_4, jac_value_4, recall_value_4, precision_value_4])
  df = pd.DataFrame(SCORE, columns = ["Image Name", "Acc_model_1", "F1_model_1", "Jaccard_model_1", "Recall_model_1", "Precision_model_1"
                                      ,"Acc_model_2", "F1_model_2", "Jaccard_model_2", "Recall_model_2", "Precision_model_2",
                                     "Acc_model_3", "F1_model_3", "Jaccard_model_3", "Recall_model_3", "Precision_model_3",
                                     "Acc_model_4", "F1_model_4", "Jaccard_model_4", "Recall_model_4", "Precision_model_4"])
  df.to_csv("Files/score.csv")
  """ mean metrics values """
  score = [s[1:] for s in SCORE]
  score = np.mean(score, axis=0)

100%|██████████| 160/160 [01:39<00:00,  1.62it/s]


In [ ]:
  print("\nModel-1 Vgg19")
  print(f"dice_coef: {score[0]:0.5f}")
  print(f"F1: {score[1]:0.5f}")
  print(f"Jaccard: {score[2]:0.5f}")
  print(f"Recall: {score[3]:0.5f}")
  print(f"Precision: {score[4]:0.5f}")
  print("\nModel-2 Resnet")
  print(f"dice_coef: {score[5]:0.5f}")
  print(f"F1: {score[6]:0.5f}")
  print(f"Jaccard: {score[7]:0.5f}")
  print(f"Recall: {score[8]:0.5f}")
  print(f"Precision: {score[9]:0.5f}")
  print("\nModel-3 Inception")
  print(f"dice_coef: {score[10]:0.5f}")
  print(f"F1: {score[11]:0.5f}")
  print(f"Jaccard: {score[12]:0.5f}")
  print(f"Recall: {score[13]:0.5f}")
  print(f"Precision: {score[14]:0.5f}")
  print("\nModel-4 ENsemble")
  print(f"dice_coef: {score[15]:0.5f}")
  print(f"F1: {score[16]:0.5f}")
  print(f"Jaccard: {score[17]:0.5f}")
  print(f"Recall: {score[18]:0.5f}")
  print(f"Precision: {score[19]:0.5f}")


Model-1 Vgg19
dice_coef: 0.95971
F1: 0.95971
Jaccard: 0.92387
Recall: 0.95106
Precision: 0.97090

Model-2 Resnet
dice_coef: 0.95414
F1: 0.95414
Jaccard: 0.91525
Recall: 0.94739
Precision: 0.96555

Model-3 Inception
dice_coef: 0.96071
F1: 0.96071
Jaccard: 0.92571
Recall: 0.96575
Precision: 0.95823

Model-4 ENsemble
dice_coef: 0.95997
F1: 0.95997
Jaccard: 0.92487
Recall: 0.98093
Precision: 0.94244


In [ ]:
orig_Y=[]
pred_Y=[]
for x, y in tqdm(zip(test_x, test_y), total=160):
                ori_x, x = read_test_image(x)
                ori_y, y = read_test_mask(y)
                y_pred_1_en = model1.predict(x)[0] > 0.5
                y_pred_2_en = model2.predict(x)[0] > 0.5
                y_pred_3_en = model3.predict(x)[0] > 0.5
                y_pred_ensem=np.array([y_pred_1_en,y_pred_2_en,y_pred_3_en])
                orig_Y.append(y)
                pred_Y.append(y_pred_ensem)

100%|██████████| 160/160 [00:52<00:00,  3.05it/s]


In [ ]:
pred_Y=np.array(pred_Y)
orig_Y=np.array(orig_Y)
orig_Y=orig_Y.flatten()

In [ ]:
from keras.metrics import MeanIoU
import pandas as pd
df = pd.DataFrame([])
for w1 in range(0, 10):
    for w2 in range(0,7):
        for w3 in range(0,7):
            wts = [w1/10.,w2/10.,w3/10.]
            y_pred_ensem=np.tensordot(pred_Y,wts,axes=((1),(0)))>0.5
            y_pred_ensem = np.squeeze(y_pred_ensem, axis=-1)
            y_pred_ensem = y_pred_ensem.astype(np.int32)
            y_pred_ensem = y_pred_ensem.flatten()
            IOU=dice(orig_Y, y_pred_ensem)
            df = df.append(pd.DataFrame({'wt1':wts[0],'wt2':wts[1], 
                                         'wt3':wts[2], 'IOU':  IOU}, index=[0]), ignore_index=True)
            
max_iou_row = df.iloc[df['IOU'].idxmax()]
print("Max dice coeff of ", max_iou_row[3], " obained with w1=", max_iou_row[0],
      " w2=", max_iou_row[1], " and w3=", max_iou_row[2]) 

Max dice coeff of  0.9701632466275458  obained with w1= 0.6  w2= 0.1  and w3= 0.5


In [ ]:

weights = [0.6, 0.1, 0.5]

for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
  name = x.split("/")[-1]
  """ Exctracting the image name """
  """ Read the image and mask """
  ori_x, x = read_test_image(x)
  ori_y, y = read_test_mask(y)
  """ Predicting the mask with Model-1 """
  y_pred_1_en = model1.predict(x)[0] > 0.5
  y_pred_1 = np.squeeze(y_pred_1_en, axis=-1)
  y_pred_1 = y_pred_1.astype(np.int32)

  """ Predicting the mask with Model-2 """
  y_pred_2_en = model2.predict(x)[0] > 0.5
  y_pred_2 = np.squeeze(y_pred_2_en, axis=-1)
  y_pred_2 = y_pred_2.astype(np.int32)

  """ Predicting the mask with Model-3 """
  y_pred_3_en = model3.predict(x)[0] > 0.5
  y_pred_3 = np.squeeze(y_pred_3_en, axis=-1)
  y_pred_3 = y_pred_3.astype(np.int32)
  
  """ Predicting the mask with ensemble-Model """
  y_pred_ensem=np.array([y_pred_1_en,y_pred_2_en,y_pred_3_en])
  y_pred_ensem=np.tensordot(y_pred_ensem,weights,axes=((0),(0)))>0.5
  y_pred_ensem = np.squeeze(y_pred_ensem, axis=-1)
  y_pred_ensem = y_pred_ensem.astype(np.int32)

  """ Flatten the array """
  # for model 1
  y = y.flatten()
  y_pred_1 = y_pred_1.flatten()

  """ Calculating metrics values """
  acc_value_1 = dice(y_pred_1, y)
  f1_value_1 = f1_score(y, y_pred_1, labels=[0, 1], average="binary")
  jac_value_1 = jaccard_score(y, y_pred_1, labels=[0, 1], average="binary")
  recall_value_1 = recall_score(y, y_pred_1, labels=[0, 1], average="binary")
  precision_value_1 = precision_score(y, y_pred_1, labels=[0, 1], average="binary")
  
  # for model 2 
  y_pred_2 = y_pred_2.flatten()

  """ Calculating metrics values """
  acc_value_2 = dice(y_pred_2, y)
  f1_value_2 = f1_score(y, y_pred_2, labels=[0, 1], average="binary")
  jac_value_2 = jaccard_score(y, y_pred_2, labels=[0, 1], average="binary")
  recall_value_2 = recall_score(y, y_pred_2, labels=[0, 1], average="binary")
  precision_value_2 = precision_score(y, y_pred_2, labels=[0, 1], average="binary")

  # for model 3
  
  y_pred_3 = y_pred_3.flatten()

  """ Calculating metrics values """
  acc_value_3 = dice(y_pred_3, y)
  f1_value_3 = f1_score(y, y_pred_3, labels=[0, 1], average="binary")
  jac_value_3 = jaccard_score(y, y_pred_3, labels=[0, 1], average="binary")
  recall_value_3 = recall_score(y, y_pred_3, labels=[0, 1], average="binary")
  precision_value_3 = precision_score(y, y_pred_3, labels=[0, 1], average="binary")

  # model 4
  y_pred_ensem = y_pred_ensem.flatten()

  """ Calculating metrics values """
  acc_value_4 = dice(y_pred_ensem, y)
  f1_value_4 = f1_score(y, y_pred_ensem, labels=[0, 1], average="binary")
  jac_value_4 = jaccard_score(y, y_pred_ensem, labels=[0, 1], average="binary")
  recall_value_4 = recall_score(y, y_pred_ensem, labels=[0, 1], average="binary")
  precision_value_4 = precision_score(y, y_pred_ensem, labels=[0, 1], average="binary")

  SCORE.append([name, acc_value_1, f1_value_1, jac_value_1, recall_value_1, precision_value_1,
                acc_value_2, f1_value_2, jac_value_2, recall_value_2, precision_value_2
               , acc_value_3, f1_value_3, jac_value_3, recall_value_3, precision_value_3
               , acc_value_4, f1_value_4, jac_value_4, recall_value_4, precision_value_4])
  df = pd.DataFrame(SCORE, columns = ["Image Name", "Acc_model_1", "F1_model_1", "Jaccard_model_1", "Recall_model_1", "Precision_model_1"
                                      ,"Acc_model_2", "F1_model_2", "Jaccard_model_2", "Recall_model_2", "Precision_model_2",
                                     "Acc_model_3", "F1_model_3", "Jaccard_model_3", "Recall_model_3", "Precision_model_3",
                                     "Acc_model_4", "F1_model_4", "Jaccard_model_4", "Recall_model_4", "Precision_model_4"])
  df.to_csv("Files/score.csv")
  """ mean metrics values """
  score = [s[1:] for s in SCORE]
  score = np.mean(score, axis=0)

100%|██████████| 160/160 [01:42<00:00,  1.56it/s]


In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
# Dice similarity function
def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice


def dice_loss(y_true, y_pred):
    return 1.0 - dice(y_true, y_pred)

In [ ]:
  print("\nModel-1 Vgg19")
  print(f"dice_coef: {score[0]:0.5f}")
  print(f"F1: {score[1]:0.5f}")
  print(f"Jaccard: {score[2]:0.5f}")
  print(f"Recall: {score[3]:0.5f}")
  print(f"Precision: {score[4]:0.5f}")
  print("\nModel-2 Resnet")
  print(f"dice_coef: {score[5]:0.5f}")
  print(f"F1: {score[6]:0.5f}")
  print(f"Jaccard: {score[7]:0.5f}")
  print(f"Recall: {score[8]:0.5f}")
  print(f"Precision: {score[9]:0.5f}")
  print("\nModel-3 Inception")
  print(f"dice_coef: {score[10]:0.5f}")
  print(f"F1: {score[11]:0.5f}")
  print(f"Jaccard: {score[12]:0.5f}")
  print(f"Recall: {score[13]:0.5f}")
  print(f"Precision: {score[14]:0.5f}")
  print("\nModel-4 ENsemble")
  print(f"dice_coef: {score[15]:0.5f}")
  print(f"F1: {score[16]:0.5f}")
  print(f"Jaccard: {score[17]:0.5f}")
  print(f"Recall: {score[18]:0.5f}")
  print(f"Precision: {score[19]:0.5f}")


Model-1 Vgg19
dice_coef: 0.95971
F1: 0.95971
Jaccard: 0.92387
Recall: 0.95106
Precision: 0.97090

Model-2 Resnet
dice_coef: 0.95414
F1: 0.95414
Jaccard: 0.91525
Recall: 0.94739
Precision: 0.96555

Model-3 Inception
dice_coef: 0.96071
F1: 0.96071
Jaccard: 0.92571
Recall: 0.96575
Precision: 0.95823

Model-4 ENsemble
dice_coef: 0.96207
F1: 0.96207
Jaccard: 0.92838
Recall: 0.97418
Precision: 0.95264


In [ ]:
def save_results(ori_x, ori_y, y_pred_1,y_pred_2,y_pred_3,y_pred_ensem, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

    y_pred_1 = np.expand_dims(y_pred_1, axis=-1)  ## (256, 256, 1)
    y_pred_1 = np.concatenate([y_pred_1, y_pred_1, y_pred_1], axis=-1) ## (256, 256, 3)

    y_pred_2 = np.expand_dims(y_pred_2, axis=-1)  ## (256, 256, 1)
    y_pred_2 = np.concatenate([y_pred_2, y_pred_2, y_pred_2], axis=-1) ## (256, 256, 3)
    
    y_pred_3 = np.expand_dims(y_pred_3, axis=-1)  ## (256, 256, 1)
    y_pred_3 = np.concatenate([y_pred_3, y_pred_3, y_pred_3], axis=-1) ## (256, 256, 3)
    
    y_pred_ensem = np.expand_dims(y_pred_ensem, axis=-1)  ## (256, 256, 1)
    y_pred_ensem = np.concatenate([y_pred_ensem, y_pred_ensem, y_pred_ensem], axis=-1) ## (256, 256, 3)
    
    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred_1*255,line,y_pred_2*255,line,y_pred_3*255,line, y_pred_ensem*255], axis=1)
    cv2.imwrite(save_image_path, cat_images)